In [1]:
import pandas as pd
from transformers import pipeline

2024-02-28 18:41:45.102646: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 18:41:45.102751: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 18:41:45.253960: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
test_df = pd.read_csv("/kaggle/input/regipa/test_regipa.csv", index_col="Index")
test_df = test_df.drop(columns=['District'])

In [3]:
pipe = pipeline("text2text-generation", model="/kaggle/input/bhashamul-ipa-final-training/bytet5-banglaregional-text-to-ipa-Final/checkpoint-17000/", device=0)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# preprocess-1: sort by length
index = test_df["Contents"].str.len().sort_values(ascending=False).index
test_df = test_df.reindex(index)

In [5]:
# preprocess-2: removing english alphabets
alpha_pat = "[a-zA-Z]"
test_df["Contents"] = test_df["Contents"].str.replace(alpha_pat, "", regex=True)

In [6]:
# preprocess-3: adding period at the end of those texts not having period or '।'
indices_to_add_period = test_df[~test_df['Contents'].str.endswith('।')].index
test_df.loc[indices_to_add_period, 'Contents'] += '।'

In [7]:
%%time
texts = test_df["Contents"].tolist()
ipas = pipe(texts, max_length=512, batch_size=64)
ipas = [ipa["generated_text"] for ipa in ipas]

CPU times: user 5min 19s, sys: 223 ms, total: 5min 19s
Wall time: 5min 19s


In [8]:
# postprocess-1
test_df["ipa"] = ipas
test_df = test_df.sort_index()

In [9]:
# postprocess-2: removing period from the end of those ipas in which texts period were added in preprocess-3
test_df.loc[indices_to_add_period, 'ipa'] = test_df.loc[indices_to_add_period, 'ipa'].str.rstrip('।')

In [10]:
test_df.index.names = ['id']
test_df = test_df.drop(columns=['Contents'])
test_df.rename(columns={'ipa': 'string'},inplace=True)
test_df.head()

,string
id,
0,elɐ ʃɔbɐe̯ ʃɔbɐɾ hɐt̪ot̪ <> ɔʃɔhɐe̯ mɐnuʃ ɐce?
1,ke͡u̯ kɐɾo ɪʲɐ nɐ͡ɪ̯।
2,"elɐ o͡ɪ̯ ɟe, kɐ͡ɪ̯lkɐ bɛtɐe̯ ɪpot̪ɐɾɪ ɐnce, kʰ..."
3,ɐɾ mu͡ɪ̯ ɐgot̪ kɪ koɾcɪnu?
4,"<> ɐgeɾ kɔt̪ɐ bɐd̪ d̪e͡o̯, elɐ nɐ͡ɪ̯।"


In [11]:
test_df.to_csv("submission_bytebase.csv")